<a href="https://colab.research.google.com/github/lavanyasaxena01/urban-parking-pricing/blob/main/Lavanya's_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uploading the dataset

In [ ]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()  # Upload the file first
print(uploaded.keys())     # Check the correct filename key

Saving dataset.csv to dataset (1).csv
dict_keys(['dataset (1).csv'])


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['dataset (1).csv']))
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


# Building Model 1

In [ ]:
import numpy as np
import pandas as pd

# Step 1: Create Timestamp column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Step 2: Sort by parking lot and time
df = df.sort_values(['SystemCodeNumber', 'Timestamp'])

# Step 3: Set base price and alpha
base_price = 10.0
alpha = 2.0

# Step 4: Initialize a new column to store price
df['LinearPrice'] = np.nan

# Step 5: Simulate pricing logic for each parking lot
for lot_id, group in df.groupby('SystemCodeNumber'):
    prices = [base_price]  # Start from base
    for i in range(1, len(group)):
        occupancy = group.iloc[i]['Occupancy']
        capacity = group.iloc[i]['Capacity']
        prev_price = prices[-1]

        # Pricing formula
        new_price = prev_price + alpha * (occupancy / capacity)

        # Keep prices within $5 and $20
        new_price = max(5, min(20, new_price))
        prices.append(new_price)

    # Assign to original DataFrame
    df.loc[group.index, 'LinearPrice'] = prices



In [ ]:
df[df['SystemCodeNumber'] == 'BHMBCCMKT01'][['Timestamp', 'Occupancy', 'Capacity', 'LinearPrice']].head(10)


,Timestamp,Occupancy,Capacity,LinearPrice
0,2016-10-04 07:59:00,61,577,10.000000
1,2016-10-04 08:25:00,64,577,10.221837
2,2016-10-04 08:59:00,80,577,10.499133
3,2016-10-04 09:32:00,107,577,10.870017
4,2016-10-04 09:59:00,150,577,11.389948
5,2016-10-04 10:26:00,177,577,12.003466
6,2016-10-04 10:59:00,219,577,12.762565
7,2016-10-04 11:25:00,247,577,13.618718
8,2016-10-04 11:59:00,259,577,14.516464
9,2016-10-04 12:29:00,266,577,15.438475


In [ ]:
traffic_map = {'low': 0.3, 'medium': 0.6, 'high': 1.0}
vehicle_map = {'car': 1.0, 'bike': 0.8, 'truck': 1.5}

df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)
df['VehicleWeight'] = df['VehicleType'].map(vehicle_map)

# Handle missing values with default assumptions
df['TrafficLevel'].fillna(0.6, inplace=True)     # assume medium traffic
df['VehicleWeight'].fillna(1.0, inplace=True)    # assume car

/tmp/ipython-input-6-493226406.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TrafficLevel'].fillna(0.6, inplace=True)     # assume medium traffic
/tmp/ipython-input-6-493226406.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

In [ ]:
df['OccRatio'] = df['Occupancy'] / df['Capacity']

# Step 6: Define weights for demand formula
α = 0.5   # weight for occupancy
β = 0.3   # queue length
γ = 0.2   # traffic level (negative impact)
δ = 0.5   # special day
ε = 0.4   # vehicle weight

# Step 7: Compute raw demand score
df['RawDemand'] = (
    α * df['OccRatio'] +
    β * df['QueueLength'] -
    γ * df['TrafficLevel'] +
    δ * df['IsSpecialDay'] +
    ε * df['VehicleWeight']
)

# Step 8: Normalize demand score between 0 and 1
min_demand = df['RawDemand'].min()
max_demand = df['RawDemand'].max()
df['NormalizedDemand'] = (df['RawDemand'] - min_demand) / (max_demand - min_demand)

# Step 9: Compute Demand-Based Price
base_price = 10.0
λ = 0.5  # demand sensitivity

df['DemandPrice'] = base_price * (1 + λ * df['NormalizedDemand'])

In [ ]:
# Step 10: Clip price between $5 and $20 for realism
df['DemandPrice'] = df['DemandPrice'].clip(lower=5, upper=20)

In [ ]:
df[['Timestamp', 'SystemCodeNumber', 'Occupancy', 'QueueLength',
    'TrafficLevel', 'IsSpecialDay', 'VehicleWeight', 'NormalizedDemand',
    'DemandPrice']].head(10)

,Timestamp,SystemCodeNumber,Occupancy,QueueLength,TrafficLevel,IsSpecialDay,VehicleWeight,NormalizedDemand,DemandPrice
0,2016-10-04 07:59:00,BHMBCCMKT01,61,1,0.3,0,1.0,0.078759,10.393795
1,2016-10-04 08:25:00,BHMBCCMKT01,64,1,0.3,0,1.0,0.079243,10.396214
2,2016-10-04 08:59:00,BHMBCCMKT01,80,2,0.3,0,1.0,0.137637,10.688183
3,2016-10-04 09:32:00,BHMBCCMKT01,107,2,0.3,0,1.0,0.141989,10.709947
4,2016-10-04 09:59:00,BHMBCCMKT01,150,2,0.3,0,0.8,0.134038,10.670190
5,2016-10-04 10:26:00,BHMBCCMKT01,177,3,0.3,0,1.0,0.209089,11.045445
6,2016-10-04 10:59:00,BHMBCCMKT01,219,6,1.0,0,1.5,0.394466,11.972330
7,2016-10-04 11:25:00,BHMBCCMKT01,247,5,0.6,0,1.0,0.320840,11.604201
8,2016-10-04 11:59:00,BHMBCCMKT01,259,5,0.6,0,1.0,0.322775,11.613874
9,2016-10-04 12:29:00,BHMBCCMKT01,266,8,1.0,0,0.8,0.461579,12.307893


# Building Model 2

In [ ]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

lot_df = df[df['SystemCodeNumber'] == 'YOUR_LOT_ID']
p = figure(x_axis_type="datetime", title="Price over Time")
p.line(lot_df['Timestamp'], lot_df['LinearPrice'])
show(p)


In [ ]:
# STEP 6: Haversine function to calculate distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # radius of Earth in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)

    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# STEP 7: Compute mean lat-long for each lot
lot_locations = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].mean().reset_index()
lot_ids = lot_locations['SystemCodeNumber'].tolist()

# Distance dictionary: (lot1, lot2) → km
distances = {}
for i, row1 in lot_locations.iterrows():
    for j, row2 in lot_locations.iterrows():
        if i != j:
            dist = haversine(row1['Latitude'], row1['Longitude'],
                             row2['Latitude'], row2['Longitude'])
            distances[(row1['SystemCodeNumber'], row2['SystemCodeNumber'])] = dist

# STEP 8: Competitive Pricing Logic
df['CompetitivePrice'] = df['DemandPrice']  # initialize

radius_km = 1.0
price_decrease_factor = 0.9
price_increase_factor = 1.05

df = df.sort_values(['Timestamp', 'SystemCodeNumber'])

# Group by timestamp
for timestamp, group in df.groupby('Timestamp'):
    # Create dictionaries of prices and occupancy for fast lookup
    lot_prices = group.set_index('SystemCodeNumber')['DemandPrice'].to_dict()
    lot_occupancy = group.set_index('SystemCodeNumber')['Occupancy'].to_dict()
    lot_capacity = group.set_index('SystemCodeNumber')['Capacity'].to_dict()

    for lot in group['SystemCodeNumber'].unique():
        current_price = lot_prices[lot]
        current_occupancy = lot_occupancy[lot]
        current_capacity = lot_capacity[lot]
        is_full = current_occupancy >= current_capacity

        # Find nearby lots
        nearby_prices = []
        for other_lot in lot_ids:
            if other_lot != lot and (lot, other_lot) in distances:
                if distances[(lot, other_lot)] <= radius_km:
                    if other_lot in lot_prices:
                        nearby_prices.append(lot_prices[other_lot])

        # Apply competition logic
        if nearby_prices:
            avg_nearby_price = np.mean(nearby_prices)
            if is_full and current_price > avg_nearby_price:
                adjusted_price = current_price * price_decrease_factor
            elif current_price < avg_nearby_price:
                adjusted_price = current_price * price_increase_factor
            else:
                adjusted_price = current_price
        else:
            adjusted_price = current_price  # no competition nearby

        # Save to CompetitivePrice column
        df.loc[(df['Timestamp'] == timestamp) &
               (df['SystemCodeNumber'] == lot),
               'CompetitivePrice'] = min(20, max(5, adjusted_price))

# STEP 9: View sample output
df[['Timestamp', 'SystemCodeNumber', 'Occupancy', 'DemandPrice', 'CompetitivePrice']].head(10)


,Timestamp,SystemCodeNumber,Occupancy,DemandPrice,CompetitivePrice
0,2016-10-04 07:59:00,BHMBCCMKT01,61,10.393795,10.913485
1312,2016-10-04 07:59:00,BHMBCCTHL01,120,10.767918,10.767918
2624,2016-10-04 07:59:00,BHMEURBRD01,117,10.739480,10.739480
3936,2016-10-04 07:59:00,BHMMBMMBX01,264,10.802430,10.802430
5248,2016-10-04 07:59:00,BHMNCPHST01,237,10.641137,11.173193
6560,2016-10-04 07:59:00,BHMNCPNST01,249,10.862488,10.862488
7872,2016-10-04 07:59:00,Broad Street,178,10.743682,10.743682
9184,2016-10-04 07:59:00,Others-CCCPS105a,709,10.787841,10.787841
10496,2016-10-04 07:59:00,Others-CCCPS119a,195,10.376981,10.895830
11808,2016-10-04 07:59:00,Others-CCCPS135a,1081,10.753180,10.753180


# Building Model 3

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

# Initialize Bokeh in notebook
output_notebook()

# Select a parking lot to visualize
sample_lot_id = 'BHMBCCMKT01'
lot_df = df[df['SystemCodeNumber'] == sample_lot_id].copy()

# Drop any rows with missing price data
lot_df = lot_df.dropna(subset=['LinearPrice', 'DemandPrice', 'CompetitivePrice'])

# Prepare the data source for plotting
source = ColumnDataSource(data={
    'time': lot_df['Timestamp'],
    'linear_price': lot_df['LinearPrice'],
    'demand_price': lot_df['DemandPrice'],
    'competitive_price': lot_df['CompetitivePrice']
})


In [ ]:
# Create the Bokeh figure (make sure f-string is used)
p = figure(
    title=f"Price Comparison for Parking Lot: {sample_lot_id}",
    x_axis_type='datetime',
    x_axis_label='Time',
    y_axis_label='Price ($)',
)

# Add lines for each pricing model
p.line(x='time', y='linear_price', source=source, color='blue', legend_label='Linear Price', line_width=2)
p.line(x='time', y='demand_price', source=source, color='green', legend_label='Demand-Based Price', line_width=2)
p.line(x='time', y='competitive_price', source=source, color='red', legend_label='Competitive Price', line_width=2)

# Customize legend and style
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Show the plot
show(p)


# Converting csv file to json file

In [ ]:
!rm -f dataset_sample.jsonl


In [ ]:
import pandas as pd

df = pd.read_csv("dataset.csv")

df_sample = df[['LastUpdatedDate', 'LastUpdatedTime', 'SystemCodeNumber', 'Occupancy', 'Capacity',
                'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']].copy()

# Drop invalid time data
df_sample = df_sample.dropna(subset=['LastUpdatedDate', 'LastUpdatedTime'])

# Convert to datetime (naive)
df_sample['timestamp'] = pd.to_datetime(
    df_sample['LastUpdatedDate'] + ' ' + df_sample['LastUpdatedTime'],
    errors='coerce',
    dayfirst=True
)

# Clean + map values
df_sample['traffic'] = df_sample['TrafficConditionNearby'].map({'low': 0.3, 'medium': 0.6, 'high': 1.0})
df_sample['vehicle_weight'] = df_sample['VehicleType'].map({'car': 1.0, 'bike': 0.8, 'truck': 1.5})

df_sample = df_sample.dropna(subset=['timestamp', 'traffic', 'vehicle_weight'])

df_sample = df_sample.rename(columns={
    'SystemCodeNumber': 'lot_id',
    'Occupancy': 'occupancy',
    'Capacity': 'capacity',
    'QueueLength': 'queue_length',
    'IsSpecialDay': 'is_special_day'
})

# ✅ Export timestamp as naive
df_sample['timestamp'] = df_sample['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_sample = df_sample.astype({
    'lot_id': str,
    'occupancy': int,
    'capacity': int,
    'queue_length': int,
    'traffic': float,
    'is_special_day': int,
    'vehicle_weight': float
})

# Final columns
df_sample = df_sample[['timestamp', 'lot_id', 'occupancy', 'capacity',
                       'queue_length', 'traffic', 'is_special_day', 'vehicle_weight']]

df_sample.to_json("dataset_sample.jsonl", orient='records', lines=True)


# Real-Time Stimulation using pathway

In [ ]:
import pathway as pw
from pathway.internals.dtype import DATE_TIME_NAIVE

# ✅ Schema using naive timestamp format
class ParkingEvent(pw.Schema):
    timestamp: DATE_TIME_NAIVE
    lot_id: str
    occupancy: int
    capacity: int
    queue_length: int
    traffic: float
    is_special_day: int
    vehicle_weight: float

# ✅ Load cleaned dataset in static mode
input_stream = pw.io.jsonlines.read(
    "dataset_sample.jsonl",
    schema=ParkingEvent,
    mode="static"
)

# ✅ Pricing logic: Demand-Based Model
@pw.udf
def compute_price(occupancy, capacity, queue_length, traffic, is_special_day, vehicle_weight):
    occ_ratio = occupancy / capacity
    raw_demand = (
        0.5 * occ_ratio +
        0.3 * queue_length -
        0.2 * traffic +
        0.5 * is_special_day +
        0.4 * vehicle_weight
    )
    norm_demand = max(0, min(1, (raw_demand - 0.5) / 1.5))
    demand_price = 10 * (1 + 0.5 * norm_demand)
    return min(20, max(5, demand_price))

# ✅ Apply logic and output
output = input_stream.select(
    timestamp=input_stream.timestamp,
    lot_id=input_stream.lot_id,
    occupancy=input_stream.occupancy,
    price=compute_price(
        input_stream.occupancy,
        input_stream.capacity,
        input_stream.queue_length,
        input_stream.traffic,
        input_stream.is_special_day,
        input_stream.vehicle_weight
    )
)

pw.io.jsonlines.write(output, "realtime_output.jsonl")

pw.run()


PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input           208                                     
    FileSystem…            0             0            0    output          208          0                          
    FileSystem…            0             0            0                                                            
    FileSystem…            0             0            0     Above you can see the latency of input and output      
    FileSystem…     finished             0            0    operators. The latency is measured as the difference    
    FileSystem…     finished             0            0   between the time when the operator processed the data    
    FileSystem…     finished             0            0        and the time when pathway acquired the data.        
    FileSystem…     finished             0            0                                                            
    FileSystem…     finished             0            0                                                            
    FileSystem…     finished             0            0                                                            
    FileSystem…     finished             0            0                                                            
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

In [ ]:
import pandas as pd
df_out = pd.read_json("realtime_output.jsonl", lines=True)
df_out.head()


""


In [ ]:
from pathway.internals.dtype import DATE_TIME_NAIVE

class ParkingEvent(pw.Schema):
    timestamp: DATE_TIME_NAIVE   # ✅ FINAL FIX HERE
    lot_id: str
    occupancy: int
    capacity: int
    queue_length: int
    traffic: float
    is_special_day: int
    vehicle_weight: float


In [ ]:
# ✅ Step 2: Read your dataset_sample.jsonl in STATIC mode
input_stream = pw.io.jsonlines.read(
    "dataset_sample.jsonl",
    schema=ParkingEvent,
    mode="static"   # static = test mode, not live stream
)





In [ ]:
# ✅ Step 3: Define demand-based pricing function
@pw.udf
def compute_price(occupancy, capacity, queue_length, traffic, is_special_day, vehicle_weight):
    occ_ratio = occupancy / capacity
    raw_demand = (
        0.5 * occ_ratio +
        0.3 * queue_length -
        0.2 * traffic +
        0.5 * is_special_day +
        0.4 * vehicle_weight
    )
    norm_demand = max(0, min(1, (raw_demand - 0.5) / 1.5))
    demand_price = 10 * (1 + 0.5 * norm_demand)
    return min(20, max(5, demand_price))

In [ ]:
# ✅ Step 4: Apply logic to generate output stream
output = input_stream.select(
    timestamp=input_stream.timestamp,
    lot_id=input_stream.lot_id,
    occupancy=input_stream.occupancy,
    price=compute_price(
        input_stream.occupancy,
        input_stream.capacity,
        input_stream.queue_length,
        input_stream.traffic,
        input_stream.is_special_day,
        input_stream.vehicle_weight
    )
)

In [ ]:
# ✅ Step 5: Write output to JSONL
pw.io.jsonlines.write(output, "realtime_output.jsonl")


In [ ]:
# ✅ Step 6: Run the simulation
pw.run()

Output()

ERROR:pathway_engine.connectors:Parse error: failed to create a field "timestamp" with type DateTimeUtc from json payload: "2016-10-04T07:59:00Z"
ERROR:pathway_engine.connectors:Parse error: failed to create a field "timestamp" with type DateTimeUtc from json payload: "2016-10-04T08:25:00Z"
ERROR:pathway_engine.connectors:Parse error: failed to create a field "timestamp" with type DateTimeUtc from json payload: "2016-10-04T08:59:00Z"
ERROR:pathway_engine.connectors:Parse error: failed to create a field "timestamp" with type DateTimeUtc from json payload: "2016-10-04T09:32:00Z"
ERROR:pathway_engine.connectors:Parse error: failed to create a field "timestamp" with type DateTimeUtc from json payload: "2016-10-04T09:59:00Z"
ERROR:pathway_engine.connectors:Parse error: failed to create a field "timestamp" with type DateTimeUtc from json payload: "2016-10-04T10:26:00Z"
ERROR:pathway_engine.connectors:Parse error: failed to create a field "timestamp" with type DateTimeUtc from json payload: "

KeyboardInterrupt: 

In [1]:
!tail -n 10 realtime_output.jsonl


tail: cannot open 'realtime_output.jsonl' for reading: No such file or directory


In [2]:
!pip install nbstripout
!nbstripout pricing_models.ipynb


Could not strip 'pricing_models.ipynb': file not found
